In [ ]:
# Load frame1 and frame2:

In [ ]:
from PIL import Image

frame1_path = "frame1.jpg"
frame2_path = "frame2.jpg"

frame1 = Image.open(frame1_path).convert("L")  # Convert to grayscale
frame2 = Image.open(frame2_path).convert("L")  # Convert to grayscale

In [ ]:
# Choose random areas of size 16x16 in frame2:

In [ ]:
import random

# Define function to select random 16x16 blocks
def select_random_block(image):
    width, height = image.size
    x = random.randint(0, width - 16)
    y = random.randint(0, height - 16)
    return image.crop((x, y, x + 16, y + 16))

# Select random blocks from frame2
block2 = select_random_block(frame2)

In [ ]:
# Find the most similar area in frame1 (Motion estimation):

In [ ]:
# Define function to find most similar area using Sum of Absolute Differences (SAD)
def motion_estimation(block, image):
    min_sad = float('inf')
    best_match = None
    
    for y in range(image.size[1] - 16):
        for x in range(image.size[0] - 16):
            region = image.crop((x, y, x + 16, y + 16))
            sad = sum(abs(p1 - p2) for p1, p2 in zip(block.getdata(), region.getdata()))
            if sad < min_sad:
                min_sad = sad
                best_match = region
    
    return best_match

# Find the most similar area in frame1
best_match = motion_estimation(block2, frame1)


In [ ]:
# Find the difference of the selected block with the most similar area in frame1:

In [ ]:
# Calculate the difference between the selected block and the best match
difference = Image.eval(ImageChops.difference(block2, best_match), lambda x: abs(x))

In [ ]:
# Use DCT to find the discrete cosine transform coefficients of the difference:

In [ ]:
import numpy as np
from scipy.fft import dct

# Convert image to numpy array
difference_array = np.array(difference)

# Apply 2D DCT
dct_coeffs_difference = dct(dct(difference_array.T, norm='ortho').T, norm='ortho')

In [ ]:
# Round the coefficients to their nearest integer:

In [ ]:
dct_coeffs_difference_rounded = np.round(dct_coeffs_difference)

In [ ]:
# Repeat steps 5-6 for the selected area:

In [ ]:
# Convert block2 to numpy array
block2_array = np.array(block2)

# Apply 2D DCT
dct_coeffs_block2 = dct(dct(block2_array.T, norm='ortho').T, norm='ortho')

# Round the coefficients
dct_coeffs_block2_rounded = np.round(dct_coeffs_block2)


In [ ]:
# Compare the coefficients in both cases and elaborate on the results:

In [ ]:
# Compute Mean Squared Error (MSE)
mse = np.mean((dct_coeffs_difference_rounded - dct_coeffs_block2_rounded) ** 2)

print("Mean Squared Error (MSE):", mse)

# Visualize Coefficients
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.hist(dct_coeffs_difference_rounded.flatten(), bins=50, color='blue', alpha=0.5, label='Difference Coefficients')
plt.hist(dct_coeffs_block2_rounded.flatten(), bins=50, color='red', alpha=0.5, label='Selected Area Coefficients')
plt.title('DCT Coefficients Distribution')
plt.xlabel('Coefficient Value')
plt.ylabel('Frequency')
plt.legend()

plt.subplot(1, 2, 2)
plt.imshow(dct_coeffs_difference_rounded, cmap='viridis', interpolation='nearest')
plt.title('DCT Coefficients of Difference')
plt.colorbar()

plt.tight_layout()
plt.show()

# Calculate Compression Ratio
num_nonzero_difference = np.count_nonzero(dct_coeffs_difference_rounded)
num_nonzero_block2 = np.count_nonzero(dct_coeffs_block2_rounded)

compression_ratio = num_nonzero_block2 / num_nonzero_difference
print("Compression Ratio:", compression_ratio)
